In [12]:
_include_('curriculum_vqa')

import numpy as np
import torch
from cvqa.model_dev.misc import Vocabulary

N = 100

prompt_vocab = Vocabulary()
X_all_str = []
X_all = []
Y_all = []
for i in range(N):
    a, b = np.random.randint(20, size=2)
    op = np.random.choice(['+', '-'])
    prompt = f'{a} {op} {b}'
    
    X_all_str.append(prompt)
    prompt_tokens = prompt.strip().split(' ')
    X_all.append(prompt_vocab.encode(prompt_tokens))
    Y_all.append(eval(prompt))
    
prompt_vocab.build()

X_all_str[:5]

['6 - 2', '17 + 17', '14 + 16', '11 - 7', '5 + 9']

In [9]:
import torch.nn as nn

class OperatorModule(nn.Module):
    
    def __init__(self, d=1):
        super().__init__()
        
    def forward(self, inputs, seed, context=None):
        inputs_vec = torch.stack(inputs)
        return torch.dot(inputs_vec, seed)
        
OperatorModule()(
    [ torch.tensor(35), torch.tensor(15) ], 
    torch.tensor([1, 1])  # +
)

tensor(50)

In [5]:
from cvqa.model_dev import parent
from cvqa.model_dev.programs import ProgramSpec, Seq2ConstTreeModel, Seq2VecsLSTM

modules_dict = {
    'O': OperatorModule()
}
program_spec = ProgramSpec(modules_dict)
seq2tree = Seq2ConstTreeModel(program_spec.vocab, 'O')

seeder_args = Seq2VecsLSTM.args(prompt_vocab, program_spec.vocab)
seeder_args['d_target'] = 2
seeder_model = Seq2VecsLSTM(seeder_args)

model = parent.MyModel(seq2tree, seeder_model, program_spec)